In [ ]:
import open3d as o3d
import numpy as np
import math
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
import math
from functools import reduce
import os
from tqdm import tqdm
import sys
sys.path.append('../')



In [ ]:
def points2pcd(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    return pcd

# Test run

In [ ]:
pc_path = os.path.join('..','data','dataset','heap','train','complete','heap2_6.ply')
# pc_path = 'existing.ply'
pcd = o3d.io.read_point_cloud(pc_path)
# pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()

In [ ]:
o3d.visualization.draw_geometries([pcd, axes])

# Generating variants

In [ ]:
def gen_single_pcd_variant(pc_path, axis = 1, max_mode = "max", margin = 0.01):
    pcd = o3d.io.read_point_cloud(pc_path)
    pcd_pts =  np.asarray(pcd.points)
    base_min = np.min(pcd_pts[:,axis])
    if max_mode == "min":
        if base_min < 0:
            base_max = -margin*abs(base_min)
        else:
            base_max = (1+margin)*base_min
    else:
        base_max = abs(np.random.randn())*np.max(pcd_pts[:,axis])
    base_idx = np.where((pcd_pts[:,axis] >= base_min) & (pcd_pts[:,axis] <= base_max))[0]
    base_pts = pcd_pts[base_idx,:]

    mean_x, std_x = np.random.randint(low=1,high=16)*np.mean(base_pts[:,0]), max(0.5,abs(0.5*np.random.randn()))*np.std(base_pts[:,0])
    mean_y, std_y = np.random.randint(low=1,high=16)*np.mean(base_pts[:,1]), max(0.2,abs(0.5*np.random.randn())*np.std(base_pts[:,1]))
    mean_z, std_z = np.random.randint(low=1,high=3)*np.mean(base_pts[:,2]), max(0.05,abs(0.5*np.random.randn()))*np.std(base_pts[:,2])

    noise_x = 0 + mean_x + std_x*np.random.randn(base_pts[:,0].shape[0])
    noise_y = 0 + mean_y + std_z*np.random.randn(base_pts[:,0].shape[0])
    noise_z = 0 #+ mean_z + std_z*np.random.randn(base_pts[:,0].shape[0])

    base_pts_noise = np.zeros_like(base_pts)
    base_pts_noise[:,0] =  base_pts[:,0] + noise_x
    base_pts_noise[:,1] =  base_pts[:,1] + noise_y
    base_pts_noise[:,2] =  base_pts[:,2] + noise_z

    pcd_pts[base_idx,:] = base_pts_noise
    pcd_noise = o3d.geometry.PointCloud()
    pcd_noise.points = o3d.utility.Vector3dVector(pcd_pts)
    
    return pcd_noise

def gen_pcd_variants(pc_path, num_variants = 5, axis = 1, max_mode = "max", margin = 0.01):
    pcd_variants = []
    for i in range(num_variants):
        pcd_variant = gen_single_pcd_variant(pc_path, axis = axis, max_mode = max_mode, margin = margin)
        pcd_variants.append(pcd_variant)
    return pcd_variants

In [ ]:
pc_path = os.path.join('..','data','dataset','heap','train','complete','heap2.ply')
test_pcd = gen_single_pcd_variant(pc_path, axis = 1, max_mode = "max", margin = 0.8)
o3d.visualization.draw_geometries([test_pcd])

In [ ]:
pc_path = os.path.join('..','data','dataset','heap','train','complete','heap2.ply')
num_variants = 10
max_mode = "max" # "max" or "min"
margin = 0.01 #between 0 and 1
axis = 1
pcd_variants = gen_pcd_variants(pc_path, num_variants, axis, max_mode, margin)


In [ ]:
dir_path = f"{os.sep}".join(pc_path.split(os.sep)[:-1])
fname = pc_path.split(os.sep)[-1]
for i in range(num_variants):
    variant_fname = f"{fname.split('.')[0]}_{i}.{fname.split('.')[1]}"
    save_path = os.path.join(dir_path, variant_fname)
    o3d.io.write_point_cloud(save_path, pcd_variants[i])
    # o3d.visualization.draw_geometries([pcd_variants[i]])

In [ ]:
test_pc_var_path = os.path.join('..','data','dataset','heap','train','complete','heap_7.ply')
test_pcd_var = o3d.io.read_point_cloud(test_pc_var_path)
o3d.visualization.draw_geometries([test_pcd_var])

# Clustering and joining

In [ ]:
def find_clusters(pcd, eps=0.2, min_points=10, remove_noise=True, show_pcd=True):
    labels = np.array(pcd.cluster_dbscan(eps=0.2, min_points=10, print_progress=False))
    max_label = labels.max()
    # print(f"point cloud has {max_label + 1} clusters")
    pcd_points = pcd.points
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    if remove_noise:
        noise_idx = labels[np.where(labels < 0)[0]]
        labels = np.delete(labels, noise_idx)
        colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
        pcd_points_modified = np.delete(pcd_points, noise_idx, axis=0)
    else:
        pcd_points_modified = pcd_points
        colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
        colors[labels < 0] = 0
    pcd_mod = o3d.geometry.PointCloud()
    pcd_mod.points = o3d.utility.Vector3dVector(pcd_points_modified)
    pcd_mod.colors = o3d.utility.Vector3dVector(colors[:, :3])
    if show_pcd:
        o3d.visualization.draw_geometries([pcd_mod])
    return pcd_mod, labels

def shifter(source, target, axis):
    sp, tp = source[:,axis], target[:,axis]
    max_sp, max_tp = np.max(sp), np.max(tp)
    min_sp, min_tp = np.min(sp), np.min(tp)
    if( max_tp <= min_sp) or (min_sp <= max_tp <= max_sp):
        t = np.max(sp) - np.max(tp)
    elif (max_sp <= min_tp) or (min_sp <= min_tp <= max_sp):
        t = np.min(sp) - np.min(tp)
    else:
        t = 0
    return t
    
def cluster_join(pc_path, pc_len_thresh = 100000, remove_noise=True, show_pcd=True):
    pcd = o3d.io.read_point_cloud(pc_path)
    if len(np.asarray(pcd.points)) > pc_len_thresh:
        pcd = pcd.voxel_down_sample(voxel_size=0.05)
    pcd_clustered, labels = find_clusters(pcd, eps=0.2, min_points=10, remove_noise=remove_noise, show_pcd=show_pcd)
    if labels.max() == 1:
        points = np.asarray(pcd_clustered.points)
        cluster_0, cluster_1 = points[np.where(labels==0)[0],:], points[np.where(labels==1)[0],:]
        if len(cluster_0) < len(cluster_1):
            cluster_0, cluster_1 = cluster_1, cluster_0
        translation_vec = np.asarray([shifter(cluster_1, cluster_0, 0),shifter(cluster_1, cluster_0, 1),0])
        new_cluster_1 = cluster_1 - translation_vec
        joint_cluster = np.vstack((cluster_0, new_cluster_1))
        joint_pcd = points2pcd(joint_cluster)
    elif labels.max() == 0:
        joint_pcd = pcd
    else:
        print(f"Number of clusters = {labels.max()+1}. Returned original point cloud for {pc_path}")
        joint_pcd = pcd
    return joint_pcd

In [ ]:
pc_path = os.path.join('..','data','dataset','heap','train','complete','heap2_6.ply')
joint_pcd = cluster_join(pc_path, pc_len_thresh = 100000, remove_noise=True, show_pcd=True)

In [ ]:
from utils.plyfile import quick_save_ply_file, load_ply

pc_dir_path = os.path.join('..','data','dataset','heap','train','complete')
pc_save_dir_path = os.path.join('..','data','dataset','heap','train','complete_processed')
# files = os.listdir(pc_dir_path)
# for i in tqdm(range(len(files))):
for f in tqdm(os.listdir(pc_dir_path)):
    pc_path = os.path.join(pc_dir_path, f)
    try:
        joint_pcd = cluster_join(pc_path, pc_len_thresh = 100000, remove_noise=True, show_pcd=False)
        quick_save_ply_file(joint_pcd.points, os.path.join(pc_save_dir_path, f))
    except:
        print(f)

In [ ]:
o3d.visualization.draw_geometries([joint_pcd])
# o3d.visualization.draw_geometries([pcd])

# Paritioning into Existing and Missing

In [ ]:
class HyperPlane(object):

    def __init__(self, params, bias):
        self.params = params
        self.bias = bias

    def check_point(self, point):
        return np.sign(np.dot(point, self.params) + self.bias)

    @staticmethod
    def get_plane_from_3_points(points):
        cp = np.cross(points[1] - points[0], points[2] - points[0])
        return HyperPlane(cp, np.dot(cp, points[0]))

    @staticmethod
    def get_random_plane():
        return HyperPlane.get_plane_from_3_points(np.random.rand(3, 3))

    def __str__(self):
        return "Plane A={}, B={}, C={}, D={}".format(*self.params, self.bias)


class SlicedDatasetGenerator(object):
    @staticmethod
    def generate_item(points,  target_partition_points_lb=1024, target_partition_points_ub=2048):
        while True:
            under = HyperPlane.get_random_plane().check_point(points) > 0
            points_under_plane = points[under]
            points_above_plane = points[~under]

            if target_partition_points_ub >= len(points_under_plane) >= target_partition_points_lb:
                return points_under_plane, points_above_plane
            if target_partition_points_ub >= len(points_above_plane) >= target_partition_points_lb:
                return points_above_plane, points_under_plane

In [ ]:
from utils.plyfile import quick_save_ply_file, load_ply

pc_dir_path = os.path.join('..','data','dataset','heap','train','complete_processed')
part_save_dir_path = os.path.join('..','data','dataset','heap','train')
num_part_variants = 10
for f in tqdm(os.listdir(pc_dir_path)):
    pc_path = os.path.join(pc_dir_path, f)
    complete = load_ply(pc_path)
    for npv in range(num_part_variants):
        existing, missing = SlicedDatasetGenerator.generate_item(complete, 
                            target_partition_points_lb=10000, target_partition_points_ub = 50000)
        f_save = f"{f.split('.')[0]}_{npv}.{f.split('.')[1]}"
        quick_save_ply_file(complete, os.path.join(part_save_dir_path, 'complete', f_save))
        quick_save_ply_file(existing, os.path.join(part_save_dir_path, 'existing', f_save))
        quick_save_ply_file(missing, os.path.join(part_save_dir_path, 'missing', f_save))

In [ ]:
part_save_dir_path = os.path.join('..','data','dataset','custom','train')
fname = 'heap_2_9.ply'
test_epc_path = os.path.join(part_save_dir_path, 'existing', fname)
test_mpc_path = os.path.join(part_save_dir_path, 'missing', fname)

epcd = o3d.io.read_point_cloud(test_epc_path)
epcd.paint_uniform_color([0.1, 0.706, 0]) # green
mpcd = o3d.io.read_point_cloud(test_mpc_path)
mpcd.paint_uniform_color([1, 0.706, 0]) # yellow
o3d.visualization.draw_geometries([epcd,mpcd])

# Junk

In [ ]:
import copy
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def perform_icp(source_pts, target_pts, threshold = 0.02):
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(source_pts)

    target = o3d.geometry.PointCloud()
    target.points = o3d.utility.Vector3dVector(target_pts)

    trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                            [-0.139, 0.967, -0.215, 0.7],
                            [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
    draw_registration_result(source, target, trans_init)

In [ ]:
# translation_vec = cluster_1[np.where(cluster_1==np.min(cluster_1[:,0]))[0][0],:] - cluster_0[np.where(cluster_0==np.max(cluster_0[:,0]))[0][0],:]
# # translation_vec[0] = 0
# translation_vec[2] = 0

# print(f"cluster_0 limits in X: [{np.min(cluster_0[:,0])},{np.max(cluster_0[:,0])}]")
# print(f"cluster_1 limits in X: [{np.min(cluster_1[:,0])},{np.max(cluster_1[:,0])}]")
# print(f"max(c1) > max(c2) > min(c1) => Required subraction in X of c1: {np.max(cluster_1[:,0]) - np.max(cluster_0[:,0])}")
# Xt_c1 = np.max(cluster_1[:,0]) - np.max(cluster_0[:,0])
# print("===============================")
# print(f"cluster_0 limits in Y: [{np.min(cluster_0[:,1])},{np.max(cluster_0[:,1])}]")
# print(f"cluster_1 limits in Y: [{np.min(cluster_1[:,1])},{np.max(cluster_1[:,1])}]")
# print(f"min(c1) > max(c2) => Required subraction in Y of c1: {np.min(cluster_1[:,1]) - np.max(cluster_0[:,1])}")
# Yt_c1 = np.min(cluster_1[:,1]) - np.max(cluster_0[:,1])
# print("===============================")
# print(f"Final translation vector to be substracted from c1: [{Xt_c1},{Yt_c1},0]")